# กรมบังคับคดี

In [ ]:
#กรมบังคับคดี
from urllib.parse import quote
from bs4 import BeautifulSoup
import requests
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import csv
import sys
import pandas as pd
from datetime import datetime
from csv import DictWriter
import json
import os.path

def deed_no_from_url(i):
    i = i.split('&')[-2].split('=')[-1].replace('%20',',').replace('-',',').replace('%',',').replace('.','').replace('(',',').replace(')',',').split(',')
    d = []
    for ii in i:
        if ii != '' and len(ii) >2:
            d.append(ii)
    d = str(d).replace('[','').replace(']','').replace(' ','').replace("'",'')
    return d
    
def read_table_data(province,page):
    prov_percen = find_percen_province(province)

    print('page : ',page)
    url = 'http://asset.led.go.th/newbidreg/asset_search_province.asp?search=ok&search_asset_type_id=&search_tumbol=&search_ampur=&search_province=' + prov_percen + '&search_sub_province=&search_price_begin=&search_price_end=&search_bid_date=&search_rai=&search_quaterrai=&search_wa=&search_rai_if=1&search_quaterrai_if=1&search_wa_if=1&search_status=1&search_person1=&page='+str(page)
    driver.get(url)
    
    max_row = len(driver.find_element_by_xpath('/html/body/div[4]/div/div[2]/div[2]/table').text.split('\n'))-12
    for row in range(1,max_row+1):
                
        lot = driver.find_element_by_xpath(f'/html/body/div[4]/div/div[2]/div[2]/table/tbody/tr[{row}]/td[1]').text
        sell_order = driver.find_element_by_xpath(f'/html/body/div[4]/div/div[2]/div[2]/table/tbody/tr[{row}]/td[2]').text
        case_id = driver.find_element_by_xpath(f'/html/body/div[4]/div/div[2]/div[2]/table/tbody/tr[{row}]/td[3]').text
        type_prop= driver.find_element_by_xpath(f'/html/body/div[4]/div/div[2]/div[2]/table/tbody/tr[{row}]/td[4]').text
        size1 = driver.find_element_by_xpath(f'/html/body/div[4]/div/div[2]/div[2]/table/tbody/tr[{row}]/td[5]').text
        size2 = driver.find_element_by_xpath(f'/html/body/div[4]/div/div[2]/div[2]/table/tbody/tr[{row}]/td[6]').text
        size3 = driver.find_element_by_xpath(f'/html/body/div[4]/div/div[2]/div[2]/table/tbody/tr[{row}]/td[7]').text
        price = driver.find_element_by_xpath(f'/html/body/div[4]/div/div[2]/div[2]/table/tbody/tr[{row}]/td[8]').text
        tumbon = driver.find_element_by_xpath(f'/html/body/div[4]/div/div[2]/div[2]/table/tbody/tr[{row}]/td[9]').text.strip()
        aumper = driver.find_element_by_xpath(f'/html/body/div[4]/div/div[2]/div[2]/table/tbody/tr[{row}]/td[10]').text.strip()
        page_row = str(page)+'/'+str(row)
        
        df = pd.read_csv(f'{path}/{file_name}', delimiter ='|')
        
        if not page_row in list(df['page']):
            driver.find_element_by_xpath(f'/html/body/div[4]/div/div[2]/div[2]/table/tbody/tr[{row}]').click()

            driver.switch_to.window(driver.window_handles[1])
            clicked_url = driver.current_url
            print('clicked_url',clicked_url)

            Law_Court_ID = clicked_url.split('&')[-3].split('=')[-1]
            deed_no = deed_no_from_url(clicked_url)
            addrno = clicked_url.split('&')[-1].split('=')[-1]        

            bit_time = []
            for i in range(6):
                try:
                    bit_time.append(driver.find_element_by_xpath(f'/html/body/div[1]/div/div/div[6]/div/table/tbody/tr[{i+1}]').text.strip())
                except:
                    bit_time.append(driver.find_element_by_xpath(f'/html/body/div[1]/div/div/div[4]/div/table/tbody/tr[{i+1}]').text.strip())

            dict_data = {
                'case_id': case_id,
                'page': page_row,
                'lot': lot,
                'sell_order': sell_order,
                'type_prop': type_prop,
                'size1' : size1,
                'size2' : size2,
                'size3' : size3,
                'price' : float(price.replace(',','')),
                'tumbon' : tumbon,
                'aumper': aumper,
                'Law_Court_ID': Law_Court_ID,
                'deed_no': deed_no,
                'addrno': addrno,
                'bit_time1': bit_time[0],
                'bit_time2': bit_time[1],
                'bit_time3': bit_time[2],
                'bit_time4': bit_time[3],
                'bit_time5': bit_time[4],
                'bit_time6': bit_time[5],
                'link': clicked_url,
                'timestamp' : str(datetime.now()),
                'province': province

            }

            driver.close()
            driver.switch_to.window(driver.window_handles[0])

            print(dict_data)
            print('\n\n\n')

            with open(f'{path}/{file_name}', 'a', encoding='UTF8') as f:

                data = []
                for i in header:
                    data.append(dict_data[i])

                writer = csv.writer(f,delimiter ='|')
                writer.writerow(data)
        else:
            print(case_id,page_row,'exist data--->pass')

def find_percen_province(province):
    url = 'http://asset.led.go.th/newbidreg/'
    driver.get(url)
 
    driver.find_element_by_xpath('/html/body/div[4]/div/div/div/div/div[3]/div/div[1]/table/tbody/tr/td[2]/select').send_keys(province)

    prov_percen = driver.current_url.split('=')[-1]

    return prov_percen

def find_maxpage(province):
    prov_percen = find_percen_province(province)
    
    page = 1
    url = 'http://asset.led.go.th/newbidreg/asset_search_province.asp?search=ok&search_asset_type_id=&search_tumbol=&search_ampur=&search_province=' + prov_percen + '&search_sub_province=&search_price_begin=&search_price_end=&search_bid_date=&search_rai=&search_quaterrai=&search_wa=&search_rai_if=1&search_quaterrai_if=1&search_wa_if=1&search_status=1&search_person1=&page='+str(page)

    driver.get(url)
    max_page = driver.find_element(By.XPATH, '/html/body/div[4]/div/div[2]/table[1]/tbody/tr/td[2]/div').text.split('/')[-1]
    print('max_page : ',max_page)
    return int(max_page)

In [ ]:
if not os.path.exists(path):
    os.makedirs(path) 

if not os.path.exists(f'{path}/{file_name}'):
    with open(f'{path}/{file_name}', 'w', encoding='UTF8') as f:
        writer = csv.writer(f,delimiter ='|')
        writer.writerow(header)

max_page = find_maxpage(province)

for page in range(1,max_page+1):
    read_table_data(province,page)
    

# ค้นหา GPS
## find gps from exist data

In [ ]:
import numpy as np
import pandas as pd

def find_gps_from_exist_file(path,deed_no):
    for file in os.listdir(path):
        if '.csv' in file:
            df2 = pd.read_csv(f'{path}/{file}', delimiter ='|')
            for index, row in df2.iterrows():
                if row['deed_no']==deed_no and str(row['lat'])!='nan':
                    if 'eva_unit' in row.keys():
                        return row['lat'],row['long'],row['eva_unit']
                    elif 'eval_unit_price' in row.keys():
                        return row['lat'],row['long'],row['eval_unit_price']
    return np.nan,np.nan,np.nan  

#make gps column if not exist
df = pd.read_csv(f'{path}/{file_name}', delimiter ='|')
if not 'lat' in df.keys():
    for i in ['lat','long','eva_unit']:
        if not i in df.keys():
            df[i] = np.nan
df.to_csv(f'{path}/{file_name}',sep='|',index=False)

#find gps from exist data
df = pd.read_csv(f'{path}/{file_name}', delimiter ='|')
for index, row in df.iterrows():
    if str(row['lat'])=='nan':
        lat,long,eva = find_gps_from_exist_file(path,row['deed_no'])
        print('update row:',index,row['case_id'],lat,long,eva)
        df['lat'][index] = lat
        df['long'][index] = long
        df['eva_unit'][index] = eva
    else:
        print('not found',row['case_id'])
df.to_csv(f'{path}/{file_name}',sep='|',index=False)
df

# find gps from website

In [ ]:
from urllib.parse import quote
from bs4 import BeautifulSoup
import requests
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import csv
import sys
import pandas as pd
from selenium.webdriver.support.ui import Select
import time 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import keyboard

start_row = 0   #-------------------------------edit
data_path = f'{path}/{file_name}'     #-------------------------------edit

def match_aumper(aumper):
    aumper_list = ['01-เมืองนนทบุรี','02-บางกรวย','03-บางใหญ่','04-บางบัวทอง','05-ไทรน้อย','06-ปากเกร็ด','บ1-บางปลา','ป1-ตลาดขวัญ','พ1-ตลิ่งชัน','พ7-ตลาดขวัญ','พ8-บางซื่อ']
    aumper = aumper.replace(' ','').split("(")[0]
    for a in aumper_list:
        if aumper in a:
            aumper = a
    return aumper

def find_gps(province,aumper,deed_no):
    #choose province
    element= Select(driver.find_element_by_xpath('/html/body/nav/form[1]/div/select'))
    element.select_by_visible_text(province)   # ddelement.select_by_value('12')

    #choose aumper
    aumper = match_aumper(aumper)
    time.sleep(1)
    element= Select(driver.find_element_by_xpath('/html/body/nav/form[2]/div/select'))
    element.select_by_visible_text(aumper)

    #insert deed number
    driver.find_element_by_xpath('/html/body/nav/form[3]/span/input').clear()
    driver.find_element_by_xpath('/html/body/nav/form[3]/span/input').send_keys(deed_no)
    
    #click button
    driver.find_element_by_xpath('/html/body/nav/form[4]/button').click()

    #gps window
    time.sleep(4)
    
    try:
        WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[1]/div[3]/span/div/div[2]/div[2]/div/div[2]')))
    except:
        return None,None,None
    
    box = driver.find_element_by_xpath('/html/body/div[1]/div[3]/span/div/div[2]/div[2]/div/div[2]').text
    
    #sometime box show but not find deeed no ---->wait 4s and find deed no again 
    try:
        deed_no_from_box = box.split('\n')[1]
    except:
        time.sleep(4)
        box = driver.find_element_by_xpath('/html/body/div[1]/div[3]/span/div/div[2]/div[2]/div/div[2]').text
        deed_no_from_box = box.split('\n')[1]
        pass
        
    print('deed_no_from_box',deed_no_from_box,type(deed_no_from_box),'deed_no',deed_no,type(deed_no))

    if deed_no_from_box == deed_no:
        print('match.......deed_no_from_box == deed_no')
        gps = driver.find_element_by_xpath('/html/body/div[1]/div[3]/span/div/div[2]/div[2]/div/div[2]/div[10]/div[2]/a').text

        lat = gps.split(',')[0]
        long = gps.split(',')[1]
        eva_unit = driver.find_element_by_xpath('/html/body/div[1]/div[3]/span/div/div[2]/div[2]/div/div[2]/div[9]/div[2]')

        try:
            eva_unit = int(eva_unit.text.split()[0].replace(',',''))
        except:
            eva_unit = 'nodetail'
            pass
        print('lat',lat) 
        print('long',long)
        print('eva_unit',eva_unit)
        
        #print('close box window')
        driver.find_element_by_xpath('/html/body/div[1]/div[3]/span/div/div[2]/div[2]/div/div[12]/button[2]').click()
        
        return lat,long,eva_unit

#init-----------------------------------------------------
options = webdriver.ChromeOptions() 
# options.add_argument("headless")
driver = webdriver.Chrome('chromedriver',options=options)

url = 'https://landsmaps.dol.go.th/'
driver.get(url)

try:
    #close popup
    print('close popup')
    time.sleep(3)
    WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[25]/div/div/div/div[1]/button/i')))
    driver.find_element_by_xpath('/html/body/div[25]/div/div/div/div[1]/button/i').click()
    
except:
    print('no close popup-->pass')
    pass

#test--------------------------------------------------------------------------------
province = 'นนทบุรี'
aumper = 'บางใหญ่'
deed_no = '58211'
gps = find_gps(province,aumper,deed_no)
if gps:
    print('Can find GPS...Work!')
    
#loop find gps-------------------------------------------------------------
df = pd.read_csv(data_path, delimiter ='|')

for index, row in df.iterrows():
    
#     if keyboard.is_pressed("q"):
#         print("q pressed, ending loop")
#         break
    
    try:
        print(index)
    #     if index == 100:
    #         break

        province = row['province']
        aumper = row['aumper']
        deed_nos = row['deed_no'].split(',')

        if pd.isna(row['lat']) and index >= start_row:
            print('row["lat"] is Null........................................')
            
            for deed_no in deed_nos:
                print('row["lat"],province,aumper,deed_no',row['lat'],province,aumper,deed_no)
                lat,long,eva_unit = find_gps(province,aumper,deed_no)
                
                if lat:
                    df['lat'][index] = lat
                    df['long'][index] = long
                    df['eva_unit'][index] = eva_unit

                    df.to_csv(data_path,sep='|',index=False)
                    break
            else:
                print('not found GPS Data...........',row['link'])

        else:
            print('have row["lat"] pass.......................................')
            
    except:
        print('error!!!!!!!!!!!',row['link'])
        pass
    


# Test

In [ ]:
# import pandas as pd
df = pd.read_csv(f'{path}/{file_name}', delimiter ='|')
df

In [ ]:
df['lat'].isna().sum()

# Map
## Circle Layer

In [ ]:
from folium import plugins
import folium
import matplotlib.pyplot as plt
import random
import pandas as pd

def int_to_number(a):
    a = str(a).split('.')[0]
    x = [a[-9:-6],a[-6:-3],a[-3:]]
    z = ''
    for index,i in enumerate(x):
        if i != '':
            z += i 
            if index != 2:
                z+=','
    return z 

def make_circle_map(df):
    latitude = df['lat'].mean()
    longitude = df['long'].mean()

    deed_map = folium.Map(location = [latitude, longitude], zoom_start = 12)

    #creat group
    colors = [
        'red',
        'blue',
        'gray',
        'darkred',
        'lightred',
        'orange',
        'beige',
        'green',
        'darkgreen',
        'lightgreen',
        'darkblue',
        'lightblue',
        'purple',
        'darkpurple',
        'pink',
        'cadetblue',
        'lightgray',
        'black'
    ]
    list_group = df['type_prop'].unique().tolist()
#     list_colors = [ random.choice(colors) for i in range(len(list_group))]
    list_colors = ['blue','red','darkgreen','gray']
    groups = {}
    for group in list_group:
        groups[group] = folium.FeatureGroup(name=group).add_to(deed_map)

    
    folium.TileLayer(
      tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
      attr = 'Esri',
      name = 'Satellite',
      ).add_to(deed_map)
    
    folium.TileLayer('openstreetmap').add_to(deed_map)
    folium.LayerControl().add_to(deed_map)

    for index, row in df.iterrows():
        def find_last_sta(bit_texts):
            def find_avi(bit_texts):
                for i,bit_text in enumerate(bit_texts):
                    for s in ['คลิกเข้าร่วม','ขายได้']:
                        if s in bit_text:
                            return s,bit_text

                return None,None

            def find_sta(bit_texts):
                for i,bit_text in enumerate(bit_texts):
                    stas = ['ถอนการยึด','งดขายส่งประกาศมิชอบ','งดขายไม่มีผู้สู้ราคา','โจทก์แถลงงดขาย','เจ้าพนักงานงดขาย','งดขาย','ขายได้']
                    for s in stas:
                        if s in bit_text:
                            return s,bit_text

                return None,None

            a,b = find_avi(bit_texts)
            if not a:
                bit_texts.reverse()
                a,b = find_sta(bit_texts)  
            return a,b.split()[0],b.split()[1]
        
        bit_texts = [row['bit_time1'],row['bit_time2'],row['bit_time3'],row['bit_time4'],row['bit_time5'],row['bit_time6']]
        staus,staus_no,staus_date = find_last_sta(bit_texts)

        lng = row['long']
        lat = row['lat']
        group = row['type_prop']
        label = row['label']
        price = int_to_number(row['price'])
        insta_post = row['label']+'#lg=1&slide=0'
        website = row['label']
        direction = f"https://www.google.co.th/maps/search/{row['lat']},+{row['long']}"
        size = [row['size1'],row['size2'],row['size3']]

        pub_html = folium.Html(f"""<p style="text-align: center;"><span style="font-family: Didot, serif; font-size: 21px;"></span></p>
        <p style="text-align: center;"><iframe src={insta_post}embed width="380" height="220" frameborder="0" scrolling="false" allowtransparency="true"></iframe>
        <p style="text-align: center;"><a href={website} target="_blank" title="website"><span style="font-family: Didot, serif; font-size: 17px;">{group} {size[0]}ไร่ {size[1]}งาน {size[2]}ตร.วา</span></a></p>
         <p style="text-align: center;"><a href={direction} target="_blank" title="นำทาง"><span style="font-family: Didot, serif; font-size: 17px;">Status: {staus} นัด {staus_no} {staus_date}</span></a></p>
        <p style="text-align: center;"><a href={direction} target="_blank" title="นำทาง"><span style="font-family: Didot, serif; font-size: 17px;">{price} บาท</span></a></p>
        """, script=True)
        popup = folium.Popup(pub_html, max_width=700)

        if str(lat)!='nan':
            n = list_group.index(group)
            color = list_colors[n]
        
            if staus == "คลิกเข้าร่วม":
                color_border = 'yellow'
                fill_opacity = 0.8
            else:
                color_border = 'grey'
                fill_opacity = 0.2
                
            if 'onweb' in list(df.keys()):
                if row['onweb'] == False:
                    color_border = 'grey'
                    fill_opacity = 0.2
                
            if group=='ที่ดินพร้อมสิ่งปลูกสร้าง' and int(row['size1'])==0 and int(row['size2'])==0 and int(row['size3'])<40:
                color = 'purple'

            groups[group].add_child(
                folium.CircleMarker(
                     [lat, lng],
                     radius=7, 
                     color=color_border,
                     fill=True,
                     fill_color=color,
                     fill_opacity=fill_opacity,
                     popup=popup
                 )
            )
        
    return deed_map

df = pd.read_csv(f'{path}/{file_name}', delimiter ='|')
df['label'] = df['link']
deed_map = make_circle_map(df)  #lat,long,type_prop,label
deed_map.save(f"{path}/{file_name.split('.')[0]}_circle_map.html")
deed_map

## Incidents

In [ ]:
#okkkk
from folium import plugins
import folium
import matplotlib.pyplot as plt
import random
import pandas as pd

def make_incidents_map(df):
    latitude = df['lat'].mean()
    longitude = df['long'].mean()
    
    sanfran_map = folium.Map(location = [latitude, longitude], zoom_start = 12)
    incidents = plugins.MarkerCluster().add_to(sanfran_map)
    
    list_group = df['type_prop'].unique().tolist()
#     list_colors = [ random.choice(colors) for i in range(len(list_group))]
    list_colors = ['darkblue','red','green','black']
    
    
    folium.TileLayer('openstreetmap').add_to(sanfran_map)
#     folium.TileLayer('Stamen Terrain').add_to(sanfran_map)
#     folium.LayerControl(collapsed=False).add_to(sanfran_map)
#     folium.LayerControl().add_to(sanfran_map)
    
    def int_to_number(a):
        a = str(a).split('.')[0]
        x = [a[-9:-6],a[-6:-3],a[-3:]]
        z = ''
        for index,i in enumerate(x):
            if i != '':
                z += i 
                if index != 2:
                    z+=','
        return z    

    for index, row in df.iterrows():
        lng = row['long']
        lat = row['lat']
        group = row['type_prop']
        label = row['label']
        price = int_to_number(row['price'])
        
        
        insta_post = row['label']+'#lg=1&slide=0'
        website = row['label']
#         direction = f"https://www.google.co.th/maps/@{row['lat']},{row['long']},15z"
#         https://www.google.co.th/maps/search/13.662452,+100.293810
        direction = f"https://www.google.co.th/maps/search/{row['lat']},+{row['long']}"
        size = [row['size1'],row['size2'],row['size3']]
    
        
#         pub_html = folium.Html(f"""<p style="text-align: center;"><span style="font-family: Didot, serif; font-size: 21px;">{name}</span></p>
#         <p style="text-align: center;"><iframe src={insta_post}embed width="240" height="290" frameborder="0" scrolling="auto" allowtransparency="true"></iframe>
#         <p style="text-align: center;"><a href={website} target="_blank" title="{name} Website"><span style="font-family: Didot, serif; font-size: 17px;">{name} Website</span></a></p>
#         <p style="text-align: center;"><a href={directions} target="_blank" title="Directions to {name}"><span style="font-family: Didot, serif; font-size: 17px;">Directions to {name}</span></a></p>
#         """, script=True)

        pub_html = folium.Html(f"""<p style="text-align: center;"><span style="font-family: Didot, serif; font-size: 21px;"></span></p>
        <p style="text-align: center;"><iframe src={insta_post}embed width="380" height="220" frameborder="0" scrolling="false" allowtransparency="true"></iframe>
        <p style="text-align: center;"><a href={website} target="_blank" title="website"><span style="font-family: Didot, serif; font-size: 17px;">{group} {size[0]}ไร่ {size[1]}งาน {size[2]}ตร.วา</span></a></p>
        <p style="text-align: center;"><a href={direction} target="_blank" title="นำทาง"><span style="font-family: Didot, serif; font-size: 17px;">{price} บาท</span></a></p>
        """, script=True)
    
        # Create pop-up with html content
        popup = folium.Popup(pub_html, max_width=700)
    
        n = list_group.index(group)
        color = list_colors[n]
        
        if str(lat)!='nan':
            folium.Marker(
                location = [lat, lng],
                icon=folium.Icon(color=color),
                markerColor=color,
                popup=popup,
            ).add_to(incidents)
            
    return sanfran_map

df = pd.read_csv(f'{path}/{file_name}', delimiter ='|')
df['label'] = df['link']
deed_map = make_incidents_map(df)  #lat,long,type_prop,label
deed_map.save(f"{path}/{file_name.split('.')[0]}_incidents_map.html")
deed_map


# combine data and make map

In [ ]:
def find_order_file(path):
    d = []
    for file in os.listdir(path):
        if '.csv' in file:
            print(file)
            date = file.split('_')[-1].split('.')[0]
            print(date)
            my_date = datetime.strptime(date, "%d%m%Y")
            d.append(my_date)
    d.sort()
    d.reverse()

    order_file = []
    for dd in d:
        for file in os.listdir(path):
            if '.csv' in file:
                date = file.split('_')[-1].split('.')[0]
                date = datetime.strptime(date, "%d%m%Y")
                if date==dd:
                    order_file.append(file)
    return order_file

files = find_order_file(path)



combile_df = pd.read_csv(f'{path}/{files[0]}', delimiter ='|')
print(combile_df.shape,combile_df.keys())

for f in files[1:]:
    df2 = pd.read_csv(f'{path}/{f}', delimiter ='|')
    for index2, row2 in df2.iterrows():
        exist = False
        for index, row in combile_df.iterrows():
            if row2['case_id']==row['case_id']:
                exist = True
                print('exist',row2['case_id'])
        
        if not exist:
            print('not exist append',row2['case_id'])
            combile_df = combile_df.append(row2, ignore_index = True)
            
current_df = pd.read_csv(f'{path}/{files[0]}', delimiter ='|')

onweb = []
for c in list(combile_df['case_id']):
    if c in list(current_df['case_id']):
        onweb.append(True)
    else:
        onweb.append(False)
combile_df['onweb'] = onweb

combile_df.to_csv(f'{path}/combile_df_{file_name}',sep='|',index=False)


In [ ]:
combile_df['label'] = combile_df['link']

deed_map = make_circle_map(combile_df)  #lat,long,type_prop,label
deed_map.save(f"{path}/combile_{file_name.split('.')[0]}_circle_map.html")
deed_map

In [ ]:
deed_map = make_incidents_map(combile_df)  #lat,long,type_prop,label
deed_map.save(f"{path}/combile_{file_name.split('.')[0]}_incidents_map.html")
deed_map